### ResNet20，论文效果91.25%，复现效果91.78%

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from keras import layers
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

Using TensorFlow backend.
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
X_train_mean = np.mean(X_train, axis=0)
X_train -= X_train_mean
X_test -= X_train_mean

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [3]:
def ResNet20():
    inputs = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(16, 3, padding="same",
                      kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    num_filters = 16
    for stack in range(3):
        for res_block in range(3):
            if stack > 0 and res_block == 0:
                shortcut = layers.Conv2D(num_filters, 1, strides=2, padding="same",
                                         kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
                x = layers.Conv2D(num_filters, 3, strides=2, padding="same",
                                  kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
            else:
                shortcut = x
                x = layers.Conv2D(num_filters, 3, padding="same",
                                  kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
            x = layers.BatchNormalization()(x)
            x = layers.Activation("relu")(x)
            x = layers.Conv2D(num_filters, 3, padding="same",
                              kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
            x = layers.BatchNormalization()(x)
            x = layers.Add()([shortcut, x])
            x = layers.Activation("relu")(x)
        num_filters *= 2
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(10, kernel_initializer='he_normal', activation="softmax")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
# Copied from https://keras.io/examples/cifar10_resnet/
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
model = ResNet20()
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr_schedule(0)), metrics=['acc'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]     

In [6]:
datagen = ImageDataGenerator(width_shift_range=0.125, height_shift_range=0.125, horizontal_flip=True)
datagen.fit(X_train)

In [7]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), patience=5, min_lr=0.5e-6, verbose=1)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=200, verbose=1, callbacks=[checkpoint, lr_scheduler, lr_reducer])

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
Learning rate:  0.001
1563/1563 [==============================] - 43s 28ms/step - loss: 1.5982 - acc: 0.4802 - val_loss: 1.4579 - val_acc: 0.5364

Epoch 00001: val_acc improved from -inf to 0.53640, saving model to models/ResNet20.h5
Epoch 2/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 1.2078 - acc: 0.6282 - val_loss: 1.3841 - val_acc: 0.6005

Epoch 00002: val_acc improved from 0.53640 to 0.60050, saving model to models/ResNet20.h5
Epoch 3/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 1.0462 - acc: 0.6893 - val_loss: 1.0831 - val_acc: 0.6821

Epoch 00003: val_acc improved from 0.60050 to 0.68210, saving model to models/ResNet20.h5
Epoch 4/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.9471 - acc: 0.7269 - val_loss: 1.3923 - val_acc: 0.6368

Epoch 00004: val_acc did not improve from 0.6

1563/1563 [==============================] - 40s 26ms/step - loss: 0.5671 - acc: 0.8756 - val_loss: 0.6547 - val_acc: 0.8500

Epoch 00035: val_acc improved from 0.84320 to 0.85000, saving model to models/ResNet20.h5
Epoch 36/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5658 - acc: 0.8761 - val_loss: 0.7508 - val_acc: 0.8227

Epoch 00036: val_acc did not improve from 0.85000
Epoch 37/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5630 - acc: 0.8752 - val_loss: 0.7884 - val_acc: 0.8142

Epoch 00037: val_acc did not improve from 0.85000
Epoch 38/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5567 - acc: 0.8793 - val_loss: 0.7441 - val_acc: 0.8254

Epoch 00038: val_acc did not improve from 0.85000
Epoch 39/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5563 - acc: 0.8805 - val_loss: 0.6404 - val_acc

1563/1563 [==============================] - 40s 26ms/step - loss: 0.5089 - acc: 0.8960 - val_loss: 0.8083 - val_acc: 0.8113

Epoch 00072: val_acc did not improve from 0.86440

Epoch 00072: ReduceLROnPlateau reducing learning rate to 0.00031622778103685084.
Epoch 73/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 25ms/step - loss: 0.5152 - acc: 0.8939 - val_loss: 0.7720 - val_acc: 0.8190

Epoch 00073: val_acc did not improve from 0.86440
Epoch 74/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5097 - acc: 0.8950 - val_loss: 1.0654 - val_acc: 0.7526

Epoch 00074: val_acc did not improve from 0.86440
Epoch 75/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5112 - acc: 0.8961 - val_loss: 0.7369 - val_acc: 0.8297

Epoch 00075: val_acc did not improve from 0.86440
Epoch 76/200
Learning rate:  0.001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.5087 

1563/1563 [==============================] - 40s 26ms/step - loss: 0.2501 - acc: 0.9639 - val_loss: 0.4509 - val_acc: 0.9116

Epoch 00108: val_acc did not improve from 0.91370
Epoch 109/200
Learning rate:  0.0001
1563/1563 [==============================] - 40s 25ms/step - loss: 0.2468 - acc: 0.9646 - val_loss: 0.4465 - val_acc: 0.9133

Epoch 00109: val_acc did not improve from 0.91370
Epoch 110/200
Learning rate:  0.0001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.2465 - acc: 0.9638 - val_loss: 0.4389 - val_acc: 0.9137

Epoch 00110: val_acc did not improve from 0.91370
Epoch 111/200
Learning rate:  0.0001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.2386 - acc: 0.9667 - val_loss: 0.4536 - val_acc: 0.9128

Epoch 00111: val_acc did not improve from 0.91370
Epoch 112/200
Learning rate:  0.0001
1563/1563 [==============================] - 40s 26ms/step - loss: 0.2405 - acc: 0.9650 - val_loss: 0.4502 - val_acc: 0.9128

Epoch 00112: val_acc d

1563/1563 [==============================] - 40s 26ms/step - loss: 0.1922 - acc: 0.9779 - val_loss: 0.4323 - val_acc: 0.9167

Epoch 00144: val_acc did not improve from 0.91770

Epoch 00144: ReduceLROnPlateau reducing learning rate to 3.1622775802825263e-06.
Epoch 145/200
Learning rate:  1e-05
1563/1563 [==============================] - 40s 25ms/step - loss: 0.1912 - acc: 0.9782 - val_loss: 0.4308 - val_acc: 0.9172

Epoch 00145: val_acc did not improve from 0.91770
Epoch 146/200
Learning rate:  1e-05
1563/1563 [==============================] - 40s 25ms/step - loss: 0.1918 - acc: 0.9790 - val_loss: 0.4332 - val_acc: 0.9163

Epoch 00146: val_acc did not improve from 0.91770
Epoch 147/200
Learning rate:  1e-05
1563/1563 [==============================] - 40s 26ms/step - loss: 0.1888 - acc: 0.9790 - val_loss: 0.4316 - val_acc: 0.9163

Epoch 00147: val_acc did not improve from 0.91770
Epoch 148/200
Learning rate:  1e-05
1563/1563 [==============================] - 40s 26ms/step - loss: 0.1

1563/1563 [==============================] - 40s 25ms/step - loss: 0.1838 - acc: 0.9797 - val_loss: 0.4346 - val_acc: 0.9167

Epoch 00180: val_acc did not improve from 0.91770
Epoch 181/200
Learning rate:  1e-06
1563/1563 [==============================] - 40s 25ms/step - loss: 0.1855 - acc: 0.9797 - val_loss: 0.4337 - val_acc: 0.9170

Epoch 00181: val_acc did not improve from 0.91770
Epoch 182/200
Learning rate:  5e-07
1563/1563 [==============================] - 40s 26ms/step - loss: 0.1840 - acc: 0.9806 - val_loss: 0.4341 - val_acc: 0.9167

Epoch 00182: val_acc did not improve from 0.91770
Epoch 183/200
Learning rate:  5e-07
1563/1563 [==============================] - 40s 25ms/step - loss: 0.1847 - acc: 0.9799 - val_loss: 0.4349 - val_acc: 0.9168

Epoch 00183: val_acc did not improve from 0.91770
Epoch 184/200
Learning rate:  5e-07
1563/1563 [==============================] - 40s 25ms/step - loss: 0.1841 - acc: 0.9801 - val_loss: 0.4355 - val_acc: 0.9173

Epoch 00184: val_acc did n

In [8]:
test = load_model("models/ResNet20.h5")
scores = test.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 208us/step
Test loss: 0.43468228211402893
Test accuracy: 0.9178000092506409
